In [2]:
import numpy as np
import pandas as pd


#df = pd.read_csv('../data/processed/processed_data.csv')

In [4]:
df.shape


(2520798, 47)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



X = df.drop('Label', axis=1)  #
y = df['Label']

# Encode labels

encode_labels = LabelEncoder()
y_encoded = encode_labels.fit_transform(y)


X_train,X_test,y_train,y_test = train_test_split(
    X,
    y_encoded,
    test_size = 0.15,  # 15 % test
    random_state = 42,
    stratify=y_encoded
)


X_train.to_csv('split_data/X_train.csv',index=False)
pd.DataFrame(y_train,columns=['Label']).to_csv('split_data/y_train.csv')
X_test.to_csv('split_data/X_test.csv',index=False)
pd.DataFrame(y_test,columns=['Label']).to_csv('split_data/y_test.csv')


In [16]:
# savel label encoder 
import joblib
joblib.dump(encode_labels,'split_data/label_encoder/label_encoder.pkl')

['split_data/label_encoder/label_encoder.pkl']

In [3]:
# load the data
X_train = pd.read_csv('split_data/X_train.csv')
y_train = pd.read_csv('split_data/y_train.csv')
X_test = pd.read_csv('split_data/X_test.csv')
y_test = pd.read_csv('split_data/y_test')

In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

# Load label encoder
encode_labels = joblib.load('split_data/label_encoder/label_encoder.pkl')

# Normalize features 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])


num_classes = len(encode_labels.classes_)
y_train_onehot = np.eye(num_classes)[y_train['Label']]
y_test_onehot = np.eye(num_classes)[y_test['Label']]

NameError: name 'joblib' is not defined

In [ ]:
X_t